In [ ]:

import numpy as np
np.set_printoptions(precision=3)


import scipy as sp
from scipy import linalg as la
from scipy import optimize as op

import matplotlib as mpl
mpl.rc('font',**{'size':18})
mpl.rc('text',usetex=False)
from matplotlib import animation as animation
from IPython.display import HTML
mpl.rc('animation', html='jshtml')
import matplotlib.pyplot as plt

import sympy as sym
from numpy import linalg as LA
A = np.eye(2)

In [ ]:

try:
  import google.colab
  COLAB = True
  print('Colaboratory Notebook')
except:
  COLAB = False
  print('Jupyter Notebook')

# Colab notebook
if COLAB:
  # render SymPy equations nicely in Colaboratory Notebook
  def colab_latex_printer(exp,**options):
    from google.colab.output._publish import javascript
    url = "https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.3/latest.js?config=default"
    javascript(url=url)
    return sym.printing.latex(exp,**options)
  
  sym.init_printing(use_latex="mathjax",latex_printer=colab_latex_printer)

# Jupyter notebook
else:
  init_printing(use_latex='mathjax')

# Python's Control Systems Toolbox

# SciPy module that implements many of the routines in ctrl
from scipy import signal as sig

Colaboratory Notebook


In [ ]:
x11 = sym.symbols('x11')
x12 = sym.symbols('x12')
x13 = sym.symbols('x13')
x22 = sym.symbols('x22')
x23 = sym.symbols('x23')
x33 = sym.symbols('x33')
X = sym.Matrix([[x11, x12, x13], [x12, x22, x23], [x13, x23, x33]])
#alpha = sym.symbols('alpha')


#X = sym.Matrix([[1, 1], [1, alpha]])

In [ ]:
X

⎡x₁₁  x₁₂  x₁₃⎤
⎢             ⎥
⎢x₁₂  x₂₂  x₂₃⎥
⎢             ⎥
⎣x₁₃  x₂₃  x₃₃⎦

In [ ]:
k1, k2, k3= sym.symbols('k_1, k_2, k_3')
K = sym.Matrix([[k1, 0, 0], [0, k2, 0], [0, 0, k3]])
K

⎡k₁  0   0 ⎤
⎢          ⎥
⎢0   k₂  0 ⎥
⎢          ⎥
⎣0   0   k₃⎦

In [ ]:
q1, q2, q3 = sym.symbols('q_1, q_2, q_3')
Q = sym.Matrix([[q1, 0, 0], [0, q2, 0], [0, 0, q3]])
Q

⎡q₁  0   0 ⎤
⎢          ⎥
⎢0   q₂  0 ⎥
⎢          ⎥
⎣0   0   q₃⎦

In [ ]:
W = sym.simplify((2*K-K@X@K))

In [ ]:
W

⎡k₁⋅(-k₁⋅x₁₁ + 2)     -k₁⋅k₂⋅x₁₂        -k₁⋅k₃⋅x₁₃   ⎤
⎢                                                    ⎥
⎢   -k₁⋅k₂⋅x₁₂     k₂⋅(-k₂⋅x₂₂ + 2)     -k₂⋅k₃⋅x₂₃   ⎥
⎢                                                    ⎥
⎣   -k₁⋅k₃⋅x₁₃        -k₂⋅k₃⋅x₂₃     k₃⋅(-k₃⋅x₃₃ + 2)⎦

In [ ]:
Q_hat = X@Q@X
Q_hat

⎡         2         2                            ⎤
⎢   q₁⋅x₁₁  + q₂⋅x₁₂      q₁⋅x₁₁⋅x₁₂ + q₂⋅x₁₂⋅x₂₂⎥
⎢                                                ⎥
⎢                                  2         2   ⎥
⎣q₁⋅x₁₁⋅x₁₂ + q₂⋅x₁₂⋅x₂₂     q₁⋅x₁₂  + q₂⋅x₂₂    ⎦

In [ ]:
# det = sym.simplify(W[0]*W[3] - W[1]*W[2])

In [ ]:
W_inv = W.inv()#sym.Matrix([[W[3], -W[1]], [-W[2], W[0]]])
W_inv

⎡                                                                             
⎢                                                                             
⎢─────────────────────────────────────────────────────────────────────────────
⎢  2                       2              2     2          2           2      
⎢k₁ ⋅k₂⋅k₃⋅x₁₁⋅x₂₂⋅x₃₃ - k₁ ⋅k₂⋅k₃⋅x₁₁⋅x₂₃  - k₁ ⋅k₂⋅k₃⋅x₁₂ ⋅x₃₃ + 2⋅k₁ ⋅k₂⋅k₃
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢            ─────────────────────────────────────────────────────────────────
⎢                                                   2               2         
⎢            k₁⋅k₂⋅k₃⋅x₁₁⋅x₂₂⋅x₃₃ - k₁⋅k₂⋅k₃⋅x₁₁⋅x₂₃  - k₁⋅k₂⋅k₃⋅x₁₂ ⋅x₃₃ + 2⋅
⎢                                                                             
⎢                                                   

In [ ]:
X_inv = X.inv()#sym.Matrix([[W[3], -W[1]], [-W[2], W[0]]])
X_inv

⎡                                    2                                        
⎢                       x₂₂⋅x₃₃ - x₂₃                                         
⎢────────────────────────────────────────────────────────────  ───────────────
⎢                     2      2                          2                     
⎢x₁₁⋅x₂₂⋅x₃₃ - x₁₁⋅x₂₃  - x₁₂ ⋅x₃₃ + 2⋅x₁₂⋅x₁₃⋅x₂₃ - x₁₃ ⋅x₂₂  x₁₁⋅x₂₂⋅x₃₃ - x
⎢                                                                             
⎢                                                                             
⎢                     -x₁₂⋅x₃₃ + x₁₃⋅x₂₃                                      
⎢────────────────────────────────────────────────────────────  ───────────────
⎢                     2      2                          2                     
⎢x₁₁⋅x₂₂⋅x₃₃ - x₁₁⋅x₂₃  - x₁₂ ⋅x₃₃ + 2⋅x₁₂⋅x₁₃⋅x₂₃ - x₁₃ ⋅x₂₂  x₁₁⋅x₂₂⋅x₃₃ - x
⎢                                                                             
⎢                                                   

In [ ]:
sym.simplify(W_inv)

⎡                                                                             
⎢                                                                             
⎢─────────────────────────────────────────────────────────────────────────────
⎢   ⎛                                       2               2                 
⎢k₁⋅⎝k₁⋅k₂⋅k₃⋅x₁₁⋅x₂₂⋅x₃₃ - k₁⋅k₂⋅k₃⋅x₁₁⋅x₂₃  - k₁⋅k₂⋅k₃⋅x₁₂ ⋅x₃₃ + 2⋅k₁⋅k₂⋅k₃
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢  ───────────────────────────────────────────────────────────────────────────
⎢                                         2               2                   
⎢  k₁⋅k₂⋅k₃⋅x₁₁⋅x₂₂⋅x₃₃ - k₁⋅k₂⋅k₃⋅x₁₁⋅x₂₃  - k₁⋅k₂⋅k₃⋅x₁₂ ⋅x₃₃ + 2⋅k₁⋅k₂⋅k₃⋅x
⎢                                                                             
⎢                                                   

In [ ]:
(W_inv@Q)

                                                                              
                                                                              
──────────────────────────────────────────────────────────────────────────────
  2                       2              2     2          2           2       
k₁ ⋅k₂⋅k₃⋅x₁₁⋅x₂₂⋅x₃₃ - k₁ ⋅k₂⋅k₃⋅x₁₁⋅x₂₃  - k₁ ⋅k₂⋅k₃⋅x₁₂ ⋅x₃₃ + 2⋅k₁ ⋅k₂⋅k₃⋅

                             ⎛                          2                     
                          q₁⋅⎝-k₂⋅k₃⋅x₂₂⋅x₃₃ + k₂⋅k₃⋅x₂₃  + 2⋅k₂⋅x₂₂ + 2⋅k₃⋅x₃
──────────────────────────────────────────────────────────────────────────────
                2          2           2                  2       2       2   
x₁₂⋅x₁₃⋅x₂₃ - k₁ ⋅k₂⋅k₃⋅x₁₃ ⋅x₂₂ - 2⋅k₁ ⋅k₂⋅x₁₁⋅x₂₂ + 2⋅k₁ ⋅k₂⋅x₁₂  - 2⋅k₁ ⋅k₃

     ⎞                                                                        
₃ - 4⎠                                                                        
──────────────────────────────────────────────────

In [ ]:

c = (W_inv@Q)[0] + (W_inv@Q)[4] + (W_inv@Q)[8]

In [ ]:
c

                                                                              
                                                                              
──────────────────────────────────────────────────────────────────────────────
  2                       2              2     2          2           2       
k₁ ⋅k₂⋅k₃⋅x₁₁⋅x₂₂⋅x₃₃ - k₁ ⋅k₂⋅k₃⋅x₁₁⋅x₂₃  - k₁ ⋅k₂⋅k₃⋅x₁₂ ⋅x₃₃ + 2⋅k₁ ⋅k₂⋅k₃⋅

                             ⎛                          2                     
                          q₁⋅⎝-k₂⋅k₃⋅x₂₂⋅x₃₃ + k₂⋅k₃⋅x₂₃  + 2⋅k₂⋅x₂₂ + 2⋅k₃⋅x₃
──────────────────────────────────────────────────────────────────────────────
                2          2           2                  2       2       2   
x₁₂⋅x₁₃⋅x₂₃ - k₁ ⋅k₂⋅k₃⋅x₁₃ ⋅x₂₂ - 2⋅k₁ ⋅k₂⋅x₁₁⋅x₂₂ + 2⋅k₁ ⋅k₂⋅x₁₂  - 2⋅k₁ ⋅k₃

     ⎞                                                                        
₃ - 4⎠                                                                        
──────────────────────────────────────────────────

In [ ]:
c1 = sym.simplify(c.subs([(q2, 0), (q3, 0)]))
c2 = sym.simplify(c.subs([(q1, 0), (q3, 0)]))
c3 = sym.simplify(c.subs([(q1, 0), (q2, 0)]))

In [ ]:
c1.subs(q1, 1)

                                                                              
                                                                              
──────────────────────────────────────────────────────────────────────────────
   ⎛                                       2               2                  
k₁⋅⎝k₁⋅k₂⋅k₃⋅x₁₁⋅x₂₂⋅x₃₃ - k₁⋅k₂⋅k₃⋅x₁₁⋅x₂₃  - k₁⋅k₂⋅k₃⋅x₁₂ ⋅x₃₃ + 2⋅k₁⋅k₂⋅k₃⋅

                                            2                                 
                  -k₂⋅k₃⋅x₂₂⋅x₃₃ + k₂⋅k₃⋅x₂₃  + 2⋅k₂⋅x₂₂ + 2⋅k₃⋅x₃₃ - 4       
──────────────────────────────────────────────────────────────────────────────
                          2                                    2              
x₁₂⋅x₁₃⋅x₂₃ - k₁⋅k₂⋅k₃⋅x₁₃ ⋅x₂₂ - 2⋅k₁⋅k₂⋅x₁₁⋅x₂₂ + 2⋅k₁⋅k₂⋅x₁₂  - 2⋅k₁⋅k₃⋅x₁₁

                                                                              
                                                                              
──────────────────────────────────────────────────

In [ ]:
c2.subs(q2, 1)

                                                                              
                                                                              
──────────────────────────────────────────────────────────────────────────────
   ⎛                                       2               2                  
k₂⋅⎝k₁⋅k₂⋅k₃⋅x₁₁⋅x₂₂⋅x₃₃ - k₁⋅k₂⋅k₃⋅x₁₁⋅x₂₃  - k₁⋅k₂⋅k₃⋅x₁₂ ⋅x₃₃ + 2⋅k₁⋅k₂⋅k₃⋅

                                            2                                 
                  -k₁⋅k₃⋅x₁₁⋅x₃₃ + k₁⋅k₃⋅x₁₃  + 2⋅k₁⋅x₁₁ + 2⋅k₃⋅x₃₃ - 4       
──────────────────────────────────────────────────────────────────────────────
                          2                                    2              
x₁₂⋅x₁₃⋅x₂₃ - k₁⋅k₂⋅k₃⋅x₁₃ ⋅x₂₂ - 2⋅k₁⋅k₂⋅x₁₁⋅x₂₂ + 2⋅k₁⋅k₂⋅x₁₂  - 2⋅k₁⋅k₃⋅x₁₁

                                                                              
                                                                              
──────────────────────────────────────────────────

In [ ]:
c3.subs(q3, 1)

                                                                              
                                                                              
──────────────────────────────────────────────────────────────────────────────
   ⎛                                       2               2                  
k₃⋅⎝k₁⋅k₂⋅k₃⋅x₁₁⋅x₂₂⋅x₃₃ - k₁⋅k₂⋅k₃⋅x₁₁⋅x₂₃  - k₁⋅k₂⋅k₃⋅x₁₂ ⋅x₃₃ + 2⋅k₁⋅k₂⋅k₃⋅

                                            2                                 
                  -k₁⋅k₂⋅x₁₁⋅x₂₂ + k₁⋅k₂⋅x₁₂  + 2⋅k₁⋅x₁₁ + 2⋅k₂⋅x₂₂ - 4       
──────────────────────────────────────────────────────────────────────────────
                          2                                    2              
x₁₂⋅x₁₃⋅x₂₃ - k₁⋅k₂⋅k₃⋅x₁₃ ⋅x₂₂ - 2⋅k₁⋅k₂⋅x₁₁⋅x₂₂ + 2⋅k₁⋅k₂⋅x₁₂  - 2⋅k₁⋅k₃⋅x₁₁

                                                                              
                                                                              
──────────────────────────────────────────────────

In [ ]:
sym.diff(c2, k2)

                    ⎛                          2                          ⎞ ⎛ 
                 q₂⋅⎝-k₁⋅k₃⋅x₁₁⋅x₃₃ + k₁⋅k₃⋅x₁₃  + 2⋅k₁⋅x₁₁ + 2⋅k₃⋅x₃₃ - 4⎠⋅⎝-
──────────────────────────────────────────────────────────────────────────────
                                                                              
   ⎛                                       2               2                  
k₂⋅⎝k₁⋅k₂⋅k₃⋅x₁₁⋅x₂₂⋅x₃₃ - k₁⋅k₂⋅k₃⋅x₁₁⋅x₂₃  - k₁⋅k₂⋅k₃⋅x₁₂ ⋅x₃₃ + 2⋅k₁⋅k₂⋅k₃⋅

                                 2            2                               
k₁⋅k₃⋅x₁₁⋅x₂₂⋅x₃₃ + k₁⋅k₃⋅x₁₁⋅x₂₃  + k₁⋅k₃⋅x₁₂ ⋅x₃₃ - 2⋅k₁⋅k₃⋅x₁₂⋅x₁₃⋅x₂₃ + k₁
──────────────────────────────────────────────────────────────────────────────
                                                                              
                          2                                    2              
x₁₂⋅x₁₃⋅x₂₃ - k₁⋅k₂⋅k₃⋅x₁₃ ⋅x₂₂ - 2⋅k₁⋅k₂⋅x₁₁⋅x₂₂ + 2⋅k₁⋅k₂⋅x₁₂  - 2⋅k₁⋅k₃⋅x₁₁

       2                              2           

In [ ]:
sym.diff(c1, k1)

                    ⎛                          2                          ⎞ ⎛ 
                 q₁⋅⎝-k₂⋅k₃⋅x₂₂⋅x₃₃ + k₂⋅k₃⋅x₂₃  + 2⋅k₂⋅x₂₂ + 2⋅k₃⋅x₃₃ - 4⎠⋅⎝-
──────────────────────────────────────────────────────────────────────────────
                                                                              
   ⎛                                       2               2                  
k₁⋅⎝k₁⋅k₂⋅k₃⋅x₁₁⋅x₂₂⋅x₃₃ - k₁⋅k₂⋅k₃⋅x₁₁⋅x₂₃  - k₁⋅k₂⋅k₃⋅x₁₂ ⋅x₃₃ + 2⋅k₁⋅k₂⋅k₃⋅

                                 2            2                               
k₂⋅k₃⋅x₁₁⋅x₂₂⋅x₃₃ + k₂⋅k₃⋅x₁₁⋅x₂₃  + k₂⋅k₃⋅x₁₂ ⋅x₃₃ - 2⋅k₂⋅k₃⋅x₁₂⋅x₁₃⋅x₂₃ + k₂
──────────────────────────────────────────────────────────────────────────────
                                                                              
                          2                                    2              
x₁₂⋅x₁₃⋅x₂₃ - k₁⋅k₂⋅k₃⋅x₁₃ ⋅x₂₂ - 2⋅k₁⋅k₂⋅x₁₁⋅x₂₂ + 2⋅k₁⋅k₂⋅x₁₂  - 2⋅k₁⋅k₃⋅x₁₁

       2                              2           

In [ ]:
def psd(n):
    """generate PSD matrix with dimension n-by-n"""
    mat = np.random.normal(0, 1, (n, n))
    return mat.T@mat



In [ ]:
sym.simplify(c2) 

                     q₂⋅(k₁⋅x₁₁ - 2)                      
──────────────────────────────────────────────────────────
   ⎛                          2                          ⎞
k₂⋅⎝-k₁⋅k₂⋅x₁₁⋅x₂₂ + k₁⋅k₂⋅x₁₂  + 2⋅k₁⋅x₁₁ + 2⋅k₂⋅x₂₂ - 4⎠

In [ ]:
sym.simplify(c1) 

                     q₁⋅(k₂⋅x₂₂ - 2)                      
──────────────────────────────────────────────────────────
   ⎛                          2                          ⎞
k₁⋅⎝-k₁⋅k₂⋅x₁₁⋅x₂₂ + k₁⋅k₂⋅x₁₂  + 2⋅k₁⋅x₁₁ + 2⋅k₂⋅x₂₂ - 4⎠

In [ ]:
d1 = sym.simplify(sym.diff(c1, k1)) 

In [ ]:
d2 = sym.simplify(sym.diff(c2, k2)) 

In [ ]:
d3 = sym.simplify(sym.diff(c3, k3)) 

In [ ]:
d1

   ⎛                          2                          ⎞ ⎛                  
q₁⋅⎝-k₂⋅k₃⋅x₂₂⋅x₃₃ + k₂⋅k₃⋅x₂₃  + 2⋅k₂⋅x₂₂ + 2⋅k₃⋅x₃₃ - 4⎠⋅⎝-k₁⋅k₂⋅k₃⋅x₁₁⋅x₂₂⋅
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                                              
                                                                              

                      2               2                                       
x₃₃ + k₁⋅k₂⋅k₃⋅x₁₁⋅x₂₃  + k₁⋅k₂⋅k₃⋅x₁₂ ⋅x₃₃ - 2⋅k₁⋅k₂⋅k₃⋅x₁₂⋅x₁₃⋅x₂₃ + k₁⋅k₂⋅k
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                2 ⎛                                       2   
                              k₁ ⋅⎝k₁⋅k₂⋅k₃⋅x₁₁⋅x₂₂⋅x₃₃ - k₁⋅k₂⋅k₃⋅x₁₁⋅x₂₃  - 

     2                                    2       

In [ ]:
from sympy.solvers import solve
k1_star = solve(d1, k1)[0]


In [ ]:
k2_star = solve(d2, k2)[0]

In [ ]:
k3_star = solve(d3, k3)[0]

In [ ]:
k1_star

                                                                           2  
                                                  k₂⋅k₃⋅x₂₂⋅x₃₃ - k₂⋅k₃⋅x₂₃  -
──────────────────────────────────────────────────────────────────────────────
                                 2            2                               
k₂⋅k₃⋅x₁₁⋅x₂₂⋅x₃₃ - k₂⋅k₃⋅x₁₁⋅x₂₃  - k₂⋅k₃⋅x₁₂ ⋅x₃₃ + 2⋅k₂⋅k₃⋅x₁₂⋅x₁₃⋅x₂₃ - k₂

                                                                          
 2⋅k₂⋅x₂₂ - 2⋅k₃⋅x₃₃ + 4                                                  
──────────────────────────────────────────────────────────────────────────
       2                              2                          2        
⋅k₃⋅x₁₃ ⋅x₂₂ - 2⋅k₂⋅x₁₁⋅x₂₂ + 2⋅k₂⋅x₁₂  - 2⋅k₃⋅x₁₁⋅x₃₃ + 2⋅k₃⋅x₁₃  + 4⋅x₁₁

In [ ]:
d_k1_star_k2 = sym.simplify(sym.diff(k1_star, k2))
d_k1_star_k2

⎛                   2        ⎞ ⎛                                 2            
⎝k₃⋅x₂₂⋅x₃₃ - k₃⋅x₂₃  - 2⋅x₂₂⎠⋅⎝k₂⋅k₃⋅x₁₁⋅x₂₂⋅x₃₃ - k₂⋅k₃⋅x₁₁⋅x₂₃  - k₂⋅k₃⋅x₁₂
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                                              
                                                                              

2                                      2                              2       
 ⋅x₃₃ + 2⋅k₂⋅k₃⋅x₁₂⋅x₁₃⋅x₂₃ - k₂⋅k₃⋅x₁₃ ⋅x₂₂ - 2⋅k₂⋅x₁₁⋅x₂₂ + 2⋅k₂⋅x₁₂  - 2⋅k₃
──────────────────────────────────────────────────────────────────────────────
                                                                              
                ⎛                                 2            2              
                ⎝k₂⋅k₃⋅x₁₁⋅x₂₂⋅x₃₃ - k₂⋅k₃⋅x₁₁⋅x₂₃  - k₂⋅k₃⋅x₁₂ ⋅x₃₃ + 2⋅k₂⋅k₃

                   2        ⎞   ⎛                 

In [ ]:
d_k1_star_k3 = sym.simplify(sym.diff(k1_star, k3))
d_k1_star_k3

⎛                   2        ⎞ ⎛                                 2            
⎝k₂⋅x₂₂⋅x₃₃ - k₂⋅x₂₃  - 2⋅x₃₃⎠⋅⎝k₂⋅k₃⋅x₁₁⋅x₂₂⋅x₃₃ - k₂⋅k₃⋅x₁₁⋅x₂₃  - k₂⋅k₃⋅x₁₂
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                                              
                                                                              

2                                      2                              2       
 ⋅x₃₃ + 2⋅k₂⋅k₃⋅x₁₂⋅x₁₃⋅x₂₃ - k₂⋅k₃⋅x₁₃ ⋅x₂₂ - 2⋅k₂⋅x₁₁⋅x₂₂ + 2⋅k₂⋅x₁₂  - 2⋅k₃
──────────────────────────────────────────────────────────────────────────────
                                                                              
                ⎛                                 2            2              
                ⎝k₂⋅k₃⋅x₁₁⋅x₂₂⋅x₃₃ - k₂⋅k₃⋅x₁₁⋅x₂₃  - k₂⋅k₃⋅x₁₂ ⋅x₃₃ + 2⋅k₂⋅k₃

                   2        ⎞   ⎛                 

In [ ]:
d_k2_star_k1 = sym.simplify(sym.diff(k2_star, k1))
d_k2_star_k1

⎛                   2        ⎞ ⎛                                 2            
⎝k₃⋅x₁₁⋅x₃₃ - k₃⋅x₁₃  - 2⋅x₁₁⎠⋅⎝k₁⋅k₃⋅x₁₁⋅x₂₂⋅x₃₃ - k₁⋅k₃⋅x₁₁⋅x₂₃  - k₁⋅k₃⋅x₁₂
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                                              
                                                                              

2                                      2                              2       
 ⋅x₃₃ + 2⋅k₁⋅k₃⋅x₁₂⋅x₁₃⋅x₂₃ - k₁⋅k₃⋅x₁₃ ⋅x₂₂ - 2⋅k₁⋅x₁₁⋅x₂₂ + 2⋅k₁⋅x₁₂  - 2⋅k₃
──────────────────────────────────────────────────────────────────────────────
                                                                              
                ⎛                                 2            2              
                ⎝k₁⋅k₃⋅x₁₁⋅x₂₂⋅x₃₃ - k₁⋅k₃⋅x₁₁⋅x₂₃  - k₁⋅k₃⋅x₁₂ ⋅x₃₃ + 2⋅k₁⋅k₃

                   2        ⎞   ⎛                 

In [ ]:
d_k2_star_k3 = sym.simplify(sym.diff(k2_star, k3))
d_k2_star_k3

⎛                   2        ⎞ ⎛                                 2            
⎝k₁⋅x₁₁⋅x₃₃ - k₁⋅x₁₃  - 2⋅x₃₃⎠⋅⎝k₁⋅k₃⋅x₁₁⋅x₂₂⋅x₃₃ - k₁⋅k₃⋅x₁₁⋅x₂₃  - k₁⋅k₃⋅x₁₂
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                                              
                                                                              

2                                      2                              2       
 ⋅x₃₃ + 2⋅k₁⋅k₃⋅x₁₂⋅x₁₃⋅x₂₃ - k₁⋅k₃⋅x₁₃ ⋅x₂₂ - 2⋅k₁⋅x₁₁⋅x₂₂ + 2⋅k₁⋅x₁₂  - 2⋅k₃
──────────────────────────────────────────────────────────────────────────────
                                                                              
                ⎛                                 2            2              
                ⎝k₁⋅k₃⋅x₁₁⋅x₂₂⋅x₃₃ - k₁⋅k₃⋅x₁₁⋅x₂₃  - k₁⋅k₃⋅x₁₂ ⋅x₃₃ + 2⋅k₁⋅k₃

                   2        ⎞   ⎛                 

In [ ]:
d_k3_star_k1 = sym.simplify(sym.diff(k3_star, k1))
d_k3_star_k1

⎛                   2        ⎞ ⎛                                 2            
⎝k₂⋅x₁₁⋅x₂₂ - k₂⋅x₁₂  - 2⋅x₁₁⎠⋅⎝k₁⋅k₂⋅x₁₁⋅x₂₂⋅x₃₃ - k₁⋅k₂⋅x₁₁⋅x₂₃  - k₁⋅k₂⋅x₁₂
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                                              
                                                                              

2                                      2                              2       
 ⋅x₃₃ + 2⋅k₁⋅k₂⋅x₁₂⋅x₁₃⋅x₂₃ - k₁⋅k₂⋅x₁₃ ⋅x₂₂ - 2⋅k₁⋅x₁₁⋅x₃₃ + 2⋅k₁⋅x₁₃  - 2⋅k₂
──────────────────────────────────────────────────────────────────────────────
                                                                              
                ⎛                                 2            2              
                ⎝k₁⋅k₂⋅x₁₁⋅x₂₂⋅x₃₃ - k₁⋅k₂⋅x₁₁⋅x₂₃  - k₁⋅k₂⋅x₁₂ ⋅x₃₃ + 2⋅k₁⋅k₂

                   2        ⎞   ⎛                 

In [ ]:
d_k3_star_k2 = sym.simplify(sym.diff(k3_star, k2))
d_k3_star_k2

⎛                   2        ⎞ ⎛                                 2            
⎝k₁⋅x₁₁⋅x₂₂ - k₁⋅x₁₂  - 2⋅x₂₂⎠⋅⎝k₁⋅k₂⋅x₁₁⋅x₂₂⋅x₃₃ - k₁⋅k₂⋅x₁₁⋅x₂₃  - k₁⋅k₂⋅x₁₂
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                                              
                                                                              

2                                      2                              2       
 ⋅x₃₃ + 2⋅k₁⋅k₂⋅x₁₂⋅x₁₃⋅x₂₃ - k₁⋅k₂⋅x₁₃ ⋅x₂₂ - 2⋅k₁⋅x₁₁⋅x₃₃ + 2⋅k₁⋅x₁₃  - 2⋅k₂
──────────────────────────────────────────────────────────────────────────────
                                                                              
                ⎛                                 2            2              
                ⎝k₁⋅k₂⋅x₁₁⋅x₂₂⋅x₃₃ - k₁⋅k₂⋅x₁₁⋅x₂₃  - k₁⋅k₂⋅x₁₂ ⋅x₃₃ + 2⋅k₁⋅k₂

                   2        ⎞   ⎛                 

In [ ]:
df_dk1

                                                                              
                                                                              
                                                                              
⎛                               ⎛                                             
⎜⎛                   2        ⎞ ⎜                                         k₂⋅x
⎜⎝k₂⋅x₂₂⋅x₃₃ - k₂⋅x₂₃  - 2⋅x₃₃⎠⋅⎜─────────────────────────────────────────────
⎜                               ⎜                                 2           
⎝                               ⎝k₁⋅k₂⋅x₁₁⋅x₂₂⋅x₃₃ - k₁⋅k₂⋅x₁₁⋅x₂₃  - k₁⋅k₂⋅x₁
                                                                              
                                                                              
                                                                              
                                                                              
────────────────────────────────────────────────────

In [ ]:
d_k1_star_k2_star = sym.simplify(d_k1_star_k2.subs([(k2, k2_star)]))

In [ ]:
d_k1_star_k3_star = sym.simplify(d_k1_star_k3.subs([(k3, k3_star)]))

In [ ]:
d_k2_star_k3_star= sym.simplify(d_k2_star_k3.subs([(k3, k3_star)]))
d_k2_star_k1_star = sym.simplify(d_k2_star_k1.subs([(k1, k1_star)]))

In [ ]:
d_k3_star_k1_star= sym.simplify(d_k3_star_k1.subs([(k1, k1_star)]))
d_k3_star_k2_star = sym.simplify(d_k3_star_k2.subs([(k2, k2_star)]))

In [ ]:
d_k1_star_k2_star

 ⎛⎛                    2        ⎞ ⎛                 ⎛                         
-⎝⎝-k₃⋅x₂₂⋅x₃₃ + k₃⋅x₂₃  + 2⋅x₂₂⎠⋅⎝- k₃⋅x₁₁⋅x₂₂⋅x₃₃⋅⎝-k₁⋅k₃⋅x₁₁⋅x₃₃ + k₁⋅k₃⋅x₁
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                                              
                                                                              

 2                          ⎞             2 ⎛                          2      
₃  + 2⋅k₁⋅x₁₁ + 2⋅k₃⋅x₃₃ - 4⎠ + k₃⋅x₁₁⋅x₂₃ ⋅⎝-k₁⋅k₃⋅x₁₁⋅x₃₃ + k₁⋅k₃⋅x₁₃  + 2⋅k
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                                              
                                                                              

                    ⎞         2     ⎛             

In [ ]:
X_ = psd(3)
X_inv = LA.inv(X_)

In [ ]:
d_k2_star_k3_star*d_k3_star_k1 + d_k2_star_k1

  ⎛⎛                    2        ⎞ ⎛                 ⎛                        
  ⎝⎝-k₁⋅x₁₁⋅x₃₃ + k₁⋅x₁₃  + 2⋅x₃₃⎠⋅⎝- k₁⋅x₁₁⋅x₂₂⋅x₃₃⋅⎝-k₁⋅k₂⋅x₁₁⋅x₂₂ + k₁⋅k₂⋅x
- ────────────────────────────────────────────────────────────────────────────
                                                                              
                                                                              
                                                                              

  2                          ⎞             2 ⎛                          2     
₁₂  + 2⋅k₁⋅x₁₁ + 2⋅k₂⋅x₂₂ - 4⎠ + k₁⋅x₁₁⋅x₂₃ ⋅⎝-k₁⋅k₂⋅x₁₁⋅x₂₂ + k₁⋅k₂⋅x₁₂  + 2⋅
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                                              
                                                                              

                     ⎞         2     ⎛            

In [ ]:
df_dk1 =  d_k1_star_k2_star * (d_k2_star_k3_star*d_k3_star_k1 + d_k2_star_k1) +  d_k1_star_k3.subs([(k3, k3_star)])*(d_k3_star_k2.subs([(k2, k2_star)])*d_k2_star_k1 + d_k3_star_k1)

In [ ]:
comp1 = d_k2_star_k3_star*d_k3_star_k1
comp2 = d_k2_star_k1

0.0808743484235492

In [ ]:
df_dk2 =  d_k2_star_k1_star * (d_k1_star_k3_star*d_k3_star_k2 + d_k1_star_k2) +  d_k2_star_k3_star *(d_k3_star_k1_star * d_k1_star_k2 + d_k3_star_k2)

In [ ]:
df_dk3 =  d_k3_star_k2_star * (d_k2_star_k1_star*d_k1_star_k3 + d_k2_star_k3) +  d_k3_star_k1_star *(d_k1_star_k2_star * d_k2_star_k3 + d_k1_star_k3)

In [ ]:
df_dk1_ = df_dk1.subs([(q1, 1), (q2, 1), (x11, X_[0][0]), (x12, X_[0][1]), (x13, X_[0][2]), (x22, X_[1][1]), (x23, X_[1][2]), (x33, X_[2][2])])

In [ ]:
df_dk2_ = df_dk2.subs([(q1, 1), (q2, 1), (x11, X_[0][0]), (x12, X_[0][1]), (x13, X_[0][2]), (x22, X_[1][1]), (x23, X_[1][2]), (x33, X_[2][2])])
df_dk3_ = df_dk3.subs([(q1, 1), (q2, 1), (x11, X_[0][0]), (x12, X_[0][1]), (x13, X_[0][2]), (x22, X_[1][1]), (x23, X_[1][2]), (x33, X_[2][2])])

In [ ]:
for k_1 in np.arange(0.1, 20, 0.3):
  for k_2 in np.arange(0.1, 20, 0.3):
    for k_3 in np.arange(0.1, 20, 0.3):
      K = np.diag(np.array([k_1, k_2, k_3]))
      if np.all(LA.eigvals(K-2*X_inv) < 0):
        df1_ = df_dk1_.subs([(k1, k_1), (k2, k_2), [k3, k_3]])
        df2_ = df_dk2_.subs([(k1, k_1), (k2, k_2), [k3, k_3]])
        df3_ = df_dk3_.subs([(k1, k_1), (k2, k_2), [k3, k_3]])
        print("df1_ is ", df1_)
        print("df2_ is ", df2_)
        print("df3_ is ", df3_)
        print('---------------------------------')

df1_ is  0.00202992716725685
df2_ is  0.174921367858526
df3_ is  0.179911563510239
---------------------------------
df1_ is  0.00336620631803972
df2_ is  0.176946624981107
df3_ is  0.242854764852254
---------------------------------
df1_ is  0.000642985899862490
df2_ is  0.209701609725135
df3_ is  0.179336446157119
---------------------------------


KeyboardInterrupt: ignored

In [ ]:
sym.simplify(d_k1_star_k2.subs([(k2, k2_star)])*d_k2_star_k1)

                            4                      
                       4⋅x₁₂                       
───────────────────────────────────────────────────
                                                  2
⎛        2                 2                    2⎞ 
⎝- k₁⋅x₁₁ ⋅x₂₂ + k₁⋅x₁₁⋅x₁₂  + 2⋅x₁₁⋅x₂₂ + 2⋅x₁₂ ⎠ 

In [ ]:
d1

q₁⋅(k₂ - 2)⋅(α⋅k₁⋅k₂ - 2⋅α⋅k₁ - k₁⋅k₂ - k₁⋅(-α⋅k₂ + 2⋅α + k₂) - 2⋅k₂ + 4)
─────────────────────────────────────────────────────────────────────────
                 2                                       2               
               k₁ ⋅(-α⋅k₁⋅k₂ + 2⋅α⋅k₁ + k₁⋅k₂ + 2⋅k₂ - 4)                

In [ ]:
d2

q₂⋅(α⋅k₁ - 2)⋅(α⋅k₁⋅k₂ - 2⋅α⋅k₁ - k₁⋅k₂ - k₂⋅(-α⋅k₁ + k₁ + 2) - 2⋅k₂ + 4)
─────────────────────────────────────────────────────────────────────────
                 2                                       2               
               k₂ ⋅(-α⋅k₁⋅k₂ + 2⋅α⋅k₁ + k₁⋅k₂ + 2⋅k₂ - 4)                

In [ ]:
sym.diff(d1, k1)

     q₁⋅(k₂ - 2)⋅(2⋅α⋅k₂ - 4⋅α - 2⋅k₂)        q₁⋅(k₂ - 2)⋅(2⋅α⋅k₂ - 4⋅α - 2⋅k₂
─────────────────────────────────────────── + ────────────────────────────────
  2                                       2                               2   
k₁ ⋅(-α⋅k₁⋅k₂ + 2⋅α⋅k₁ + k₁⋅k₂ + 2⋅k₂ - 4)                              k₁ ⋅(-

)⋅(α⋅k₁⋅k₂ - 2⋅α⋅k₁ - k₁⋅k₂ - k₁⋅(-α⋅k₂ + 2⋅α + k₂) - 2⋅k₂ + 4)   2⋅q₁⋅(k₂ - 2
─────────────────────────────────────────────────────────────── - ────────────
                                    3                                         
α⋅k₁⋅k₂ + 2⋅α⋅k₁ + k₁⋅k₂ + 2⋅k₂ - 4)                                          

)⋅(α⋅k₁⋅k₂ - 2⋅α⋅k₁ - k₁⋅k₂ - k₁⋅(-α⋅k₂ + 2⋅α + k₂) - 2⋅k₂ + 4)
───────────────────────────────────────────────────────────────
      3                                       2                
    k₁ ⋅(-α⋅k₁⋅k₂ + 2⋅α⋅k₁ + k₁⋅k₂ + 2⋅k₂ - 4)                 

In [ ]:
sym.diff(d2, k1)

α⋅q₂⋅(α⋅k₁⋅k₂ - 2⋅α⋅k₁ - k₁⋅k₂ - k₂⋅(-α⋅k₁ + k₁ + 2) - 2⋅k₂ + 4)   q₂⋅(α⋅k₁ - 
──────────────────────────────────────────────────────────────── + ───────────
            2                                       2                         
          k₂ ⋅(-α⋅k₁⋅k₂ + 2⋅α⋅k₁ + k₁⋅k₂ + 2⋅k₂ - 4)                          

2)⋅(2⋅α⋅k₂ - 4⋅α - 2⋅k₂)⋅(α⋅k₁⋅k₂ - 2⋅α⋅k₁ - k₁⋅k₂ - k₂⋅(-α⋅k₁ + k₁ + 2) - 2⋅k
──────────────────────────────────────────────────────────────────────────────
                 2                                       3                    
               k₂ ⋅(-α⋅k₁⋅k₂ + 2⋅α⋅k₁ + k₁⋅k₂ + 2⋅k₂ - 4)                     

₂ + 4)   q₂⋅(α⋅k₁ - 2)⋅(α⋅k₂ - 2⋅α - k₂⋅(1 - α) - k₂)
────── + ────────────────────────────────────────────
           2                                       2 
         k₂ ⋅(-α⋅k₁⋅k₂ + 2⋅α⋅k₁ + k₁⋅k₂ + 2⋅k₂ - 4)  

In [ ]:
sym.diff(d1, k2)

q₁⋅(k₂ - 2)⋅(2⋅α⋅k₁ - 2⋅k₁ - 4)⋅(α⋅k₁⋅k₂ - 2⋅α⋅k₁ - k₁⋅k₂ - k₁⋅(-α⋅k₂ + 2⋅α + 
──────────────────────────────────────────────────────────────────────────────
                           2                                       3          
                         k₁ ⋅(-α⋅k₁⋅k₂ + 2⋅α⋅k₁ + k₁⋅k₂ + 2⋅k₂ - 4)           

k₂) - 2⋅k₂ + 4)     q₁⋅(k₂ - 2)⋅(α⋅k₁ - k₁⋅(1 - α) - k₁ - 2)    q₁⋅(α⋅k₁⋅k₂ - 
─────────────── + ─────────────────────────────────────────── + ──────────────
                    2                                       2               2 
                  k₁ ⋅(-α⋅k₁⋅k₂ + 2⋅α⋅k₁ + k₁⋅k₂ + 2⋅k₂ - 4)              k₁ ⋅

2⋅α⋅k₁ - k₁⋅k₂ - k₁⋅(-α⋅k₂ + 2⋅α + k₂) - 2⋅k₂ + 4)
──────────────────────────────────────────────────
                                      2           
(-α⋅k₁⋅k₂ + 2⋅α⋅k₁ + k₁⋅k₂ + 2⋅k₂ - 4)            

In [ ]:
sym.diff(d2, k2)

     q₂⋅(α⋅k₁ - 2)⋅(2⋅α⋅k₁ - 2⋅k₁ - 4)        q₂⋅(α⋅k₁ - 2)⋅(2⋅α⋅k₁ - 2⋅k₁ - 4
─────────────────────────────────────────── + ────────────────────────────────
  2                                       2                              2    
k₂ ⋅(-α⋅k₁⋅k₂ + 2⋅α⋅k₁ + k₁⋅k₂ + 2⋅k₂ - 4)                             k₂ ⋅(-α

)⋅(α⋅k₁⋅k₂ - 2⋅α⋅k₁ - k₁⋅k₂ - k₂⋅(-α⋅k₁ + k₁ + 2) - 2⋅k₂ + 4)   2⋅q₂⋅(α⋅k₁ - 2
───────────────────────────────────────────────────────────── - ──────────────
                                   3                                          
⋅k₁⋅k₂ + 2⋅α⋅k₁ + k₁⋅k₂ + 2⋅k₂ - 4)                                           

)⋅(α⋅k₁⋅k₂ - 2⋅α⋅k₁ - k₁⋅k₂ - k₂⋅(-α⋅k₁ + k₁ + 2) - 2⋅k₂ + 4)
─────────────────────────────────────────────────────────────
    3                                       2                
  k₂ ⋅(-α⋅k₁⋅k₂ + 2⋅α⋅k₁ + k₁⋅k₂ + 2⋅k₂ - 4)                 

In [ ]:
cross_terms = sym.simplify(sym.diff(d2, k1) + sym.diff(d1, k2))

In [ ]:
G = sym.Matrix([[sym.diff(d1, k1), sym.diff(d1, k2)], [sym.diff(d2, k1), sym.diff(d2, k2)]]).subs([(q1, 1), (q2, 1), (x11, B[0][0]), (x12, B[0][1]), (x22, B[1][1])])

In [ ]:
G

⎡0.000245043473703217⋅(1.03048536327967⋅k₂ - 2)⋅(25.1060599375181⋅k₂ - 127.763
⎢─────────────────────────────────────────────────────────────────────────────
⎢     2                                                                       
⎢   k₁ ⋅(-0.196503430162858⋅k₁⋅k₂ + k₁ + 0.0322621564866848⋅k₂ - 0.06261545798
⎢                                                                             
⎢              0.000245043473703217⋅(31.9409945117887⋅k₁ - 2)⋅(25.106059937518
⎢              ───────────────────────────────────────────────────────────────
⎢                   2                                                         
⎣                 k₂ ⋅(-0.196503430162858⋅k₁⋅k₂ + k₁ + 0.0322621564866848⋅k₂ -

978047155)   3.83587733332437e-6⋅(1.03048536327967⋅k₂ - 2)⋅(25.1060599375181⋅k
────────── + ─────────────────────────────────────────────────────────────────
      2                                                                     2 
96328)                                             

In [ ]:
X_inv

⎡ -1      1  ⎤
⎢─────  ─────⎥
⎢1 - α  1 - α⎥
⎢            ⎥
⎢  1     -α  ⎥
⎢─────  ─────⎥
⎣1 - α  1 - α⎦

In [ ]:
X_inv_vals = np.array(X_inv.subs([(alpha, 2)])).astype(np.float64)

In [ ]:
X_inv_vals

array([[ 1., -1.],
       [-1.,  2.]])

In [ ]:
X_vals

array([[2., 1.],
       [1., 1.]])

In [ ]:
det_vals = (4*(sym.diff(d1, k1) * sym.diff(d2, k2)) -  cross_terms*cross_terms).subs([(q1, 1), (q2, 1), (x11, B[0][0]), (x12, B[0][1]), (x22, B[1][1])])

In [ ]:
for k_1 in np.arange(0.1, 20, 0.5):
  for k_2 in np.arange(0.1, 20, 0.5):
    K = np.diag(np.array([k_1, k_2]))
    if np.all(np.abs(LA.eigvals(A-B@K)) < 1):
    #if np.all(LA.eigvals(K-2*X_inv) < 0):
      J = G.subs([(k1, k_1), (k2, k_2)])
      J_ = np.array(J).astype(np.float64)
      #print(det_vals.subs([(k1, k_1), (k2, k_2)]))
      #print(LA.eigvals(J_ + J_.T))
      try:
        assert np.all(LA.eigvals(J_ + J_.T) > 0)
      except:
        print()
        print(det_vals.subs([(k1, k_1), (k2, k_2)]))
        print(LA.eigvals(J_ + J_.T))

-0.0845787138478478
[ 6.354e+01 -1.331e-03]
-0.228419026543740
[ 7.054e+01 -3.238e-03]
-0.397781016389480
[ 7.867e+01 -5.056e-03]
-0.607028257581556
[ 8.817e+01 -6.885e-03]
-0.875388504783381
[ 9.935e+01 -8.811e-03]
-1.22993329430013
[ 1.126e+02 -1.092e-02]
-1.71014318790502
[ 1.284e+02 -1.331e-02]
-2.37522020283221
[ 1.476e+02 -1.610e-02]


In [ ]:
np.all(np.abs(LA.eigvals(A-B@K)) < 1)

False